In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
import pickle
import json
import h5py
import pandas as pd


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_ref = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))
# on my system os error often pops up
# try to open the file explorer and copy paste the path again 
# https://drive.google.com/open?id=1pPf-7AmUVceVfgfmKEJ6ireEDKEJHw-7
f = h5py.File("../data/static_ml20m.hdf5", "r")
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../data/metadata.json'))

In [3]:
class Actor(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=2e-1):
        super(Actor, self).__init__()
        
        self.drop_layer = nn.Dropout(p=0.5)
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, num_actions)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        # state = self.state_rep(state)
        x = F.relu(self.linear1(state))
        x = self.drop_layer(x)
        x = F.relu(self.linear2(x))
        x = self.drop_layer(x)
        # x = torch.tanh(self.linear3(x)) # in case embeds are -1 1 normalized
        x = self.linear3(x) # in case embeds are standard scaled / wiped using PCA whitening
        # return state, x
        return x

In [4]:
ddpg = Actor(1290, 128, 256)
td3 = Actor(1290, 128, 256)
ddpg.load_state_dict(torch.load('../models/DDPG/ddpg_policy(Radam).pt'))
td3.load_state_dict(torch.load('../models/TD3/td3_policy_1.pt'))

batch_size = 5000
n_batches = (f['done'].shape[0] // batch_size) - 1

In [5]:
test_batch = [f[key][n_batches*batch_size:(n_batches+1)*batch_size] for key in
                 ['movies', 'ratings', 'done']]


movies, ratings, done = test_batch
movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
state = torch.cat([movies_tensor[:, :-1, :].view(batch_size, -1),
                       ratings[:, :-1]], 1)


In [6]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}

In [7]:
def rank(gen_action, metric):
    scores = []
    for i in movie_ref.keys():
        scores.append([i, metric(movie_ref[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [8]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [9]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,87499,11.499584,Why Did I Get Married Too?,Comedy Romance,en,2010,5.8
1,60896,11.843939,The Godless Girl,Drama,en,1929,7.4
2,5997,11.883318,Champagne for Caesar,Comedy,en,1950,6.9
3,105178,11.906794,Killers from Space,Horror Sci-Fi,en,1954,3.4
4,111931,11.972146,Raze,Horror Thriller,en,2013,5.3
5,44245,12.003018,Leprechaun in the Hood,Horror,en,2000,4.5
6,2592,12.021310,The Joyriders,,en,1999,6.0
7,31374,12.055358,Hobson's Choice,Comedy Drama,en,1954,7.3
8,54725,12.067271,Under the Yum-Yum Tree,Comedy Drama Romance,en,1963,7.1
9,105042,12.116078,The Wedding Party,Comedy Romance,en,1969,4.9


In [10]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,44245,0.672237,Leprechaun in the Hood,Horror,en,2000,4.5
1,99007,0.695122,Warm Bodies,Horror Comedy Romance,en,2013,6.3
2,109191,0.700135,Winter's Tale,Drama Fantasy Mystery Romance,en,2014,6.2
3,1450,0.704042,Prisoner of the Mountains,Drama War,ru,1996,6.6
4,90419,0.706038,A Pain in the Ass,Comedy,fr,2008,5.1
5,95201,0.708990,To Rome with Love,Romance Comedy Drama,en,2012,5.7
6,87499,0.709252,Why Did I Get Married Too?,Comedy Romance,en,2010,5.8
7,62594,0.710915,The Banishment,Drama,ru,2007,7.2
8,86813,0.713688,Persona non grata,Drama,en,2005,6.5
9,5751,0.715632,Goodbye Pork Pie,Adventure Comedy,en,1981,6.8


In [11]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,44245,0.673157,Leprechaun in the Hood,Horror,en,2000,4.5
1,99007,0.697062,Warm Bodies,Horror Comedy Romance,en,2013,6.3
2,109191,0.699397,Winter's Tale,Drama Fantasy Mystery Romance,en,2014,6.2
3,1450,0.705647,Prisoner of the Mountains,Drama War,ru,1996,6.6
4,87499,0.707329,Why Did I Get Married Too?,Comedy Romance,en,2010,5.8
5,90419,0.707510,A Pain in the Ass,Comedy,fr,2008,5.1
6,95201,0.710213,To Rome with Love,Romance Comedy Drama,en,2012,5.7
7,62594,0.711327,The Banishment,Drama,ru,2007,7.2
8,86813,0.713895,Persona non grata,Drama,en,2005,6.5
9,33164,0.716580,House of Wax,Horror Drama,en,2005,5.7


In [12]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,53974,77.666844,License to Wed,Comedy,en,2007,5.6
1,113261,78.092142,Screamtime,Horror,en,1983,4.3
2,54686,78.731304,The Last Legion,Action Adventure Fantasy War,en,2007,5.4
3,58315,79.419558,The Love Guru,Comedy Romance,en,2008,4.3
4,91852,79.962082,Cloportes,Comedy Crime,fr,1965,6.5
5,7325,80.020470,Starsky & Hutch,Comedy Crime,en,2004,5.7
6,5305,80.214477,Return of the Killer Tomatoes!,Comedy Horror Sci-Fi,en,1988,5.3
7,74156,80.269202,Edge of Darkness,Crime Drama Mystery Thriller,en,2010,6.3
8,39234,80.434849,North Country,Drama,en,2005,6.9
9,125455,80.459519,Something of Value,Drama,en,1957,5.3


In [13]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,87499,11.499584,Why Did I Get Married Too?,Comedy Romance,en,2010,5.8
1,60896,11.843939,The Godless Girl,Drama,en,1929,7.4
2,5997,11.883318,Champagne for Caesar,Comedy,en,1950,6.9
3,105178,11.906794,Killers from Space,Horror Sci-Fi,en,1954,3.4
4,111931,11.972146,Raze,Horror Thriller,en,2013,5.3
5,44245,12.003018,Leprechaun in the Hood,Horror,en,2000,4.5
6,2592,12.021310,The Joyriders,,en,1999,6.0
7,31374,12.055358,Hobson's Choice,Comedy Drama,en,1954,7.3
8,54725,12.067271,Under the Yum-Yum Tree,Comedy Drama Romance,en,1963,7.1
9,105042,12.116078,The Wedding Party,Comedy Romance,en,1969,4.9


In [14]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,44193,2.397499,She's the Man,Comedy Drama Family Romance,en,2006,6.6
1,26226,2.424193,"Hi, Mom!",Comedy Drama,en,1970,5.8
2,107115,2.444067,"A Fairly Odd Movie: Grow Up, Timmy Turner!",Family Fantasy,en,2011,4.7
3,87006,2.473881,Eyes of a Stranger,Horror Thriller,en,1981,6.0
4,113751,2.496530,A Birder's Guide to Everything,Comedy,en,2013,6.4
5,64550,2.497890,The Strange Door,Horror Thriller,en,1951,6.0
6,959,2.498150,Of Human Bondage,Drama,en,1934,6.9
7,108819,2.538244,The Grave,Comedy,en,1996,5.5
8,105238,2.542027,New Town Killers,Thriller Drama,en,2008,5.7
9,120610,2.543092,Jim Breuer: And Laughter for All,Comedy,en,2013,5.5


In [15]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,53974,0.685720,License to Wed,Comedy,en,2007,5.6
1,44245,0.696378,Leprechaun in the Hood,Horror,en,2000,4.5
2,90419,0.706331,A Pain in the Ass,Comedy,fr,2008,5.1
3,8206,0.719559,Mystery of the Wax Museum,Horror Mystery Thriller,en,1933,6.8
4,7325,0.728261,Starsky & Hutch,Comedy Crime,en,2004,5.7
5,99007,0.730873,Warm Bodies,Horror Comedy Romance,en,2013,6.3
6,87499,0.731886,Why Did I Get Married Too?,Comedy Romance,en,2010,5.8
7,54686,0.732271,The Last Legion,Action Adventure Fantasy War,en,2007,5.4
8,34336,0.732819,Must Love Dogs,Romance Comedy,en,2005,5.9
9,4228,0.735959,Heartbreakers,Crime Comedy Romance,en,2001,5.9


In [16]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,87499,102.528641,Why Did I Get Married Too?,Comedy Romance,en,2010,5.8
1,53974,103.609184,License to Wed,Comedy,en,2007,5.6
2,82055,105.302048,The Devil to Pay!,Comedy Romance,en,1930,6.7
3,126122,106.779495,Let's Get Those English Girls,Comedy,fr,1976,5.4
4,7405,107.217369,Road to Bali,Comedy,en,1952,6.0
5,44245,107.308830,Leprechaun in the Hood,Horror,en,2000,4.5
6,26080,107.532654,Eegah,Horror,en,1962,2.8
7,111931,107.713501,Raze,Horror Thriller,en,2013,5.3
8,100087,108.015640,Night of the Demons 2,Comedy Horror,en,1994,5.7
9,90419,108.067551,A Pain in the Ass,Comedy,fr,2008,5.1


# TD3

In [17]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [18]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,104215,9.453526,Burning Bright,Drama Horror Thriller,en,2012,6.1
1,33817,9.782286,My Summer of Love,Drama Thriller Romance,en,2005,6.3
2,69842,9.814475,Good Dick,Comedy Drama Romance,en,2008,6.2
3,3955,9.922800,The Ladies Man,Comedy,en,2000,5.3
4,1477,10.012519,Love Jones,Comedy Drama Romance,en,1997,8.1
5,1498,10.032861,Inventing the Abbotts,Drama Romance,en,1997,6.1
6,42013,10.109309,The Ringer,Comedy,en,2005,5.7
7,73526,10.110170,A Matter of Dignity,Drama,el,1958,7.0
8,26226,10.122808,"Hi, Mom!",Comedy Drama,en,1970,5.8
9,117354,10.141997,Parasomnia,Horror Thriller,en,2008,4.6


In [19]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,104215,0.669237,Burning Bright,Drama Horror Thriller,en,2012,6.1
1,26523,0.670015,"Silent Night, Deadly Night",Thriller Horror,en,1984,5.7
2,891,0.674674,Halloween: The Curse of Michael Myers,Horror Thriller,en,1995,5.0
3,115205,0.689683,Recipe for Love,TV Movie Drama Romance,en,2014,6.4
4,86894,0.693872,Born to Be Bad,Drama,en,1950,6.0
5,76832,0.698607,The Black Waters of Echo's Pond,Horror Thriller,en,2009,5.0
6,116501,0.706309,Die Fighting,Thriller Action,en,2014,4.4
7,63662,0.708622,Where a Good Man Goes,Action Drama Thriller,cn,1999,9.0
8,8574,0.708800,Will Vinton's Claymation Christmas Celebration,Animation Comedy,en,1987,7.4
9,74317,0.717763,Escape from Suburbia: Beyond the American Dream,Documentary,en,2007,6.0


In [20]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,104215,0.666013,Burning Bright,Drama Horror Thriller,en,2012,6.1
1,26523,0.669890,"Silent Night, Deadly Night",Thriller Horror,en,1984,5.7
2,891,0.674702,Halloween: The Curse of Michael Myers,Horror Thriller,en,1995,5.0
3,115205,0.689233,Recipe for Love,TV Movie Drama Romance,en,2014,6.4
4,86894,0.693729,Born to Be Bad,Drama,en,1950,6.0
5,76832,0.698336,The Black Waters of Echo's Pond,Horror Thriller,en,2009,5.0
6,116501,0.705524,Die Fighting,Thriller Action,en,2014,4.4
7,63662,0.706644,Where a Good Man Goes,Action Drama Thriller,cn,1999,9.0
8,8574,0.707058,Will Vinton's Claymation Christmas Celebration,Animation Comedy,en,1987,7.4
9,74317,0.717682,Escape from Suburbia: Beyond the American Dream,Documentary,en,2007,6.0


In [21]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,6822,76.147588,The Ballad of Little Jo,Romance Drama Western,en,1993,5.8
1,105713,77.253047,How to Meet Girls from a Distance,,en,2012,4.8
2,86894,77.533487,Born to Be Bad,Drama,en,1950,6.0
3,26013,77.642375,Rodan,Horror Sci-Fi,ja,1956,5.8
4,92511,78.006529,Swamp Shark,Horror Thriller,en,2011,4.3
5,614,78.296203,Loaded,Drama Mystery Thriller,en,1994,4.0
6,103665,78.355116,Ip Man: The Final Fight,Action Drama,cn,2013,6.2
7,74317,78.640469,Escape from Suburbia: Beyond the American Dream,Documentary,en,2007,6.0
8,93126,78.696694,Bag of Bones,Drama Horror Mystery Thriller,en,2011,5.8
9,116501,78.774301,Die Fighting,Thriller Action,en,2014,4.4


In [22]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,104215,9.453526,Burning Bright,Drama Horror Thriller,en,2012,6.1
1,33817,9.782286,My Summer of Love,Drama Thriller Romance,en,2005,6.3
2,69842,9.814475,Good Dick,Comedy Drama Romance,en,2008,6.2
3,3955,9.922800,The Ladies Man,Comedy,en,2000,5.3
4,1477,10.012519,Love Jones,Comedy Drama Romance,en,1997,8.1
5,1498,10.032861,Inventing the Abbotts,Drama Romance,en,1997,6.1
6,42013,10.109309,The Ringer,Comedy,en,2005,5.7
7,73526,10.110170,A Matter of Dignity,Drama,el,1958,7.0
8,26226,10.122808,"Hi, Mom!",Comedy Drama,en,1970,5.8
9,117354,10.141997,Parasomnia,Horror Thriller,en,2008,4.6


In [23]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,6907,2.043868,Holy Matrimony,Comedy,en,1994,5.1
1,71429,2.060749,World's Greatest Dad,Comedy Romance Drama,en,2009,6.6
2,113751,2.079787,A Birder's Guide to Everything,Comedy,en,2013,6.4
3,107213,2.131190,The Truth About Emanuel,Drama Thriller,en,2013,6.1
4,100194,2.147144,Jim Jefferies: Fully Functional,Comedy,en,2012,7.6
5,88129,2.149733,Drive,Drama Action Thriller Crime,en,2011,7.5
6,26287,2.153147,A Delicate Balance,Drama,en,1973,6.5
7,70571,2.172591,Just Sex and Nothing Else,Drama Comedy Romance,hu,2005,6.8
8,31867,2.183672,Man of the House,Comedy Action,en,2005,5.4
9,84242,2.193531,Welcome to the Rileys,Drama,en,2010,6.5


In [24]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,6822,0.690744,The Ballad of Little Jo,Romance Drama Western,en,1993,5.8
1,86894,0.700037,Born to Be Bad,Drama,en,1950,6.0
2,76832,0.702894,The Black Waters of Echo's Pond,Horror Thriller,en,2009,5.0
3,104215,0.703243,Burning Bright,Drama Horror Thriller,en,2012,6.1
4,92511,0.704194,Swamp Shark,Horror Thriller,en,2011,4.3
5,74317,0.711358,Escape from Suburbia: Beyond the American Dream,Documentary,en,2007,6.0
6,26523,0.715651,"Silent Night, Deadly Night",Thriller Horror,en,1984,5.7
7,891,0.717147,Halloween: The Curse of Michael Myers,Horror Thriller,en,1995,5.0
8,116501,0.717980,Die Fighting,Thriller Action,en,2014,4.4
9,63662,0.723307,Where a Good Man Goes,Action Drama Thriller,cn,1999,9.0


In [25]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,104215,87.078491,Burning Bright,Drama Horror Thriller,en,2012,6.1
1,69842,87.845474,Good Dick,Comedy Drama Romance,en,2008,6.2
2,42013,88.045212,The Ringer,Comedy,en,2005,5.7
3,26226,88.054855,"Hi, Mom!",Comedy Drama,en,1970,5.8
4,33817,89.229752,My Summer of Love,Drama Thriller Romance,en,2005,6.3
5,73526,89.283203,A Matter of Dignity,Drama,el,1958,7.0
6,121821,91.055054,The Dancer,Drama,id,2011,5.9
7,105713,91.217300,How to Meet Girls from a Distance,,en,2012,4.8
8,117354,91.267990,Parasomnia,Horror Thriller,en,2008,4.6
9,111245,91.704063,A Case of You,Comedy Romance,en,2013,5.5
